Nog een keer hetzelfde programma alleen deze keer maak ik het met CustomTkinter. Dat is een Python UI library based on Tkinter die een nieuw, modernere widgets heeft.

In [1]:
# import sys
# !{sys.executable} -m pip install customtkinter

In [4]:
import tkinter
import customtkinter
import numpy as num
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [28]:
import tkinter
import customtkinter

class App():
    def __init__(self):
        customtkinter.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
        customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"

        app = customtkinter.CTk()
        app.geometry("400x400")
        app.title("DataVisualisation App voor Biker")
        
        self.loginAttempts = 0

        def loginCheck():
            # gelezen logingegevens... Super Secure!!!
            creds = pd.read_csv('logingegevens.csv')

            # omzetten naar 2D array. [[username, password], [username, password], enz..]
            credsArr = creds.to_numpy()

            # check login input
            self.username = usernameEntry.get()
            self.password = passwordEntry.get()

            # loop series van namen en ww
            for arr in credsArr:
                # subset van naam en ww
                if arr[0] == self.username:
                    if arr[1] == self.password:
                        loginOut(True)
                        return
            
            loginOut(False)
                
        def loginOut(out):
            if out == True:
                app.destroy()
                Main()

            if self.loginAttempts >= 3:
                exitMessage = customtkinter.CTkLabel(master=loginWindow, text="U heeft 3 keer de verkeerde logingegevens ingevuld. De applicatie sluit nu.")
                exitMessage.pack()
                
            elif out == False:
                label = customtkinter.CTkLabel(master=loginWindow, text="Logingegevens zijn incorrect. Probeer opnieuw.")
                label.pack()
                self.loginAttempts = self.loginAttempts + 1
            


        loginWindow = customtkinter.CTkFrame(master=app)
        loginWindow.pack(pady=20, padx=60, fill="both", expand=True)

        usernameEntry = customtkinter.CTkEntry(master=loginWindow, placeholder_text="Please enter your username:")
        usernameEntry.pack(pady=10, padx=10)

        passwordEntry = customtkinter.CTkEntry(master=loginWindow, placeholder_text="Please enter your password", show="*")
        passwordEntry.pack()

        button_1 = customtkinter.CTkButton(master=loginWindow, text="Login!", command=loginCheck)
        button_1.pack(pady=10, padx=10)

        app.mainloop()


In [29]:
class Main():
    def __init__(self):
        main = customtkinter.CTk()
        main.geometry("600X800")
        frameSide = customtkinter.CTkFrame(master=main)
        frameSide.pack(pady=30, padx = 60, fill="both", expand=True)

        label = customtkinter.CTkLabel(master=frameSide, text="Kies hieronder data dat je wilt zien")
        label.pack(pady=10, padx=10)

        main.mainloop()

In [30]:
x = App()

CustomTkinter can adapt to the Windows 10/11 light or dark mode